This model aims to simulate a simple case of a three-lane road with one lane designated as a PUV lane. Homogeneous vehicles

In [8]:
import numpy as np
import matplotlib.pyplot as plt

In [9]:
class Vehicle:
    def __init__(self, x_position, y_position, speed, max_speed, length, width, road_designation, vehicle_type, current_lane, lane_changing_prob):
    #This specified the dimensions and attributes of vehicles. Vehicle identification is also incorporated by assigning IDs
        self.x_position = x_position #horizontal position
        self.y_position = y_position #vertical position or lane deisgnation (current lane is the same)
        self.speed = speed #number of cells the vehicle moves per timestep
        self.max_speed = max_speed #maximum speed allowed for a vehicle
        self.braking_prob = 0.01 #probability of braking
        self.length = length #x dimension of vehicle
        self.width = width #y dimension of vehicle
        self.road_designation = road_designation #identify of the road where the vehicle is
        self.vehicle_type = vehicle_type #type of vehicle(car, jeep, truck, minibus, motor)
        self.current_lane = current_lane
        self.lane_changing_prob = lane_changing_prob #probability of lane changing

    def accelerate(self):
    #Accelerate the vehicle by increments of 1 if maximum speed is not yet attained
        if self.speed < self.max_speed:
            self.speed += 1 

    def decelerate(self, gap_distance):
    #Decrease the speed of the vehicle if the gap distance is less than current speed
        if self.speed > gap_distance:
            self.speed = gap_distance

    def randomize(self, braking_prob):
    #Randomly decrease the speed of the vehicle based on braking probability
        if np.random.rand() < braking_prob:
            if self.speed > 0:
                self.speed -= 1

    def lane_changing(self): #Let's assume assume a three-lane road(3 cells wide)
        if np.random.rand() < self.lane_changing_prob:
            if self.current_lane == 1: #road width is 6
                self.current_lane += 1 #there's a chance of swtiching left or right, for cars, 
            elif self.current_lane == 2: #Lanes 0 and 1 are PUV lanes, you cannot move to occupy these lanes if you are in lane 2
                self.current_lane -= 1
            #No lane changing for PUVs(jeeps and minibuses)

In [10]:
class Road:
    def __init__(self, road_id, road_length, road_width, speed_limit, has_toll_gate = False):
        #Road features
        self.road_id = road_id
        self.road_length = road_length
        self.road_width = road_width
        self.road_occupancy = np.zeros((self.road_length, self.road_width), dtype=int) #initialize a state of road occupancy
        self.speed_limit = speed_limit #maximum speed possible

In [56]:
class IntraRoadSimulator:
    def __init__(self, road):
        self.road = road
        self.priv_vehicle_production_prob = 0.9
        self.PUV_production_prob = 0.9
        self.vehicles = [] #initialize an empty list of vehicles (road knows the vehicles in it)
        
    def update_occupancy(self):
        #Updates the state of occupancy of the road. 0 means unoccupied, 1 means occupied
        self.road.road_occupancy.fill(0)  # Reset occupancy grid
        for vehicle in self.vehicles:
            if vehicle.x_position < self.road.road_length and vehicle.y_position < self.road.road_width :  # Only update occupancy if within road length and width
                self.road.road_occupancy[vehicle.x_position, vehicle.y_position] = 1  # Assuming (y + n) lanes
    
    def produce_vehicles(self):
        #This method determines the production of new vehicles
        vehicle_type = None
        vehicle_length = 0
        vehicle_width = 0
        current_lane = 0
        lane_changing_prob = 0

        if np.random.rand() < self.priv_vehicle_production_prob: #Determines the instance/time to produce vehicles
            vehicle_type = 'private vehicle' 
            vehicle_length = 1
            vehicle_width = 1
            current_lane = np.random.choice(range(0, self.road.road_width, 1)) #pick an initial lane
            lane_changing_prob = 0.01
        elif np.random.rand() < self.PUV_production_prob: #Determines the instance/time to produce vehicles
            vehicle_type = 'PUV' 
            vehicle_length = 1
            vehicle_width = 1
            current_lane = 0 #pick an initial lane
            lane_changing_prob = 0
            
        return vehicle_type, vehicle_length, vehicle_width, current_lane, lane_changing_prob

    def road_checker(self, vehicle_type, vehicle_length, vehicle_width, current_lane, lane_changing_prob):
        #Check the availability of the first cells corresponding to vehicle dimensions
        road_portion_checked = self.road.road_occupancy[0:vehicle_length, current_lane:current_lane + vehicle_width]
        if np.sum(road_portion_checked) == 0:  # Check the availability of the portion corresponding to vehicle dimensions
            new_vehicle = Vehicle(0, current_lane, np.random.randint(1, self.road.speed_limit + 1), self.road.speed_limit, vehicle_length, vehicle_width, self.road, vehicle_type, current_lane, 0.3)
            self.vehicles.append(new_vehicle)
            self.road.road_occupancy[0:new_vehicle.length, current_lane:current_lane + new_vehicle.width] = 1  # Mark the position as occupied

    def gap_distance(self, vehicle): 
        #Determines the distance between the rear bumper of the vehicle ahead and front bumper of the follower vehicle
        current_x_position = vehicle.x_position 
        maximum_possible_distance = self.road.road_length - current_x_position - vehicle.length #DETERMINES THE CORRESPONDING DISTANCE TO BE CHECKED FOR HEADWAY
        for distance in range(1, maximum_possible_distance + 1): 
            next_x_position = current_x_position + vehicle.length + distance #prospective position of the rear bumper of the vehicle
            if np.any(self.road.road_occupancy[next_x_position , vehicle.current_lane] == 1): #checking the space 
                return distance
        return maximum_possible_distance

    def update(self): 
        #updates the road configuration based on nagel-schreckenberg rules
        vehicle_type, vehicle_length, vehicle_width, current_lane, lane_changing_prob = self.produce_vehicles()
        self.road_checker(vehicle_type, vehicle_length, vehicle_width, current_lane, lane_changing_prob)
        self.update_occupancy()
        for vehicle in self.vehicles:
            distance_to_next = self.gap_distance(vehicle)
            vehicle.accelerate()
            vehicle.decelerate(distance_to_next)
            vehicle.randomize(vehicle.braking_prob)
            vehicle.x_position += vehicle.speed
            vehicle.lane_changing()

        self.update_occupancy()

    def spatio_temporal_plot(self, num_steps):
        road_snapshots = []
        for step in range(num_steps):
            self.update()
            road_snapshots.append(self.road.road_occupancy.copy())

        spatio_temporal_matrix = np.stack(road_snapshots)
        plt.figure(figsize=(20, 16))
        plt.imshow(spatio_temporal_matrix.transpose(1, 0, 2).reshape(self.road.road_width, -1), cmap='Greys', interpolation='nearest', aspect='auto')
        plt.xlabel('Space (Position along road)')
        plt.ylabel('Time Step')
        plt.xticks(range(0, self.road.road_length, 1))
        plt.yticks(range(0, num_steps, 1))
        plt.title('Spatio-Temporal Plot of Road Occupancy')

In [57]:
road = Road(road_id=1, road_length=10, road_width=3, speed_limit=5)
simulator = IntraRoadSimulator(road)

In [58]:
num_steps = 50
simulator.spatio_temporal_plot(num_steps)

IndexError: index 10 is out of bounds for axis 0 with size 10